In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

In [94]:
df = pd.read_csv("../data/train.csv")

C:\Users\jessi\AppData\Local\Temp\ipykernel_48808\3828657111.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/train.csv")


In [95]:
df = df.drop('SSN', axis=1)

In [96]:
rules = {
    "Age": lambda x: x > 0,
    "Occupation": lambda x: x != '_______',
    "Annual_Income": lambda x: x > 0,
    'Monthly_Inhand_Salary': lambda x: x > 0,
    'Num_Bank_Accounts': lambda x: x > 0,
    'Num_Credit_Card': lambda x: x > 0,
    'Interest_Rate': lambda x: x > 0,
    'Num_of_Loan': lambda x: x > 0,
    'Changed_Credit_Limit': lambda x: x >= 0,
    'Credit_Mix': lambda x: x != '_',
    'Amount_invested_monthly': lambda x: x >= 0,
    'Payment_Behaviour': lambda x: x in ['High_spent_Small_value_payments','Low_spent_Large_value_payments','Low_spent_Medium_value_payments','Low_spent_Small_value_payments','High_spent_Medium_value_payments','High_spent_Large_value_payments']
}

def detect_all_abnormal_values(df: pd.DataFrame, rules: dict):
    ''' This function will get through each of the columns with weird values and 
    replace them with NaN. The ideia is to try and get a complete row at the end
    with reasanable values.'''

    numeric_cols = ['Age','Annual_Income','Monthly_Inhand_Salary','Num_Bank_Accounts',
    'Num_Credit_Card','Interest_Rate','Num_of_Loan','Changed_Credit_Limit','Amount_invested_monthly']

    df = df.copy()  # evita SettingWithCopyWarning
    for col, rule in rules.items():
        if col in numeric_cols:
            try:
                df[col] = df[col].apply(lambda x: x.replace('_',''))
            except:
                pass
            df[col] = pd.to_numeric(df[col], errors="coerce")
            
        mask = df[col].apply(lambda x: rule(x) if pd.notna(x) else False)
        df.loc[~mask, col] = np.nan
    
    df_no_nan_rows = df.dropna()
    return df_no_nan_rows.tail(1)

In [97]:
all_customers = df['Customer_ID'].unique().tolist()
print(f"Number of customers: {len(all_customers)}")

Number of customers: 12500


In [98]:
final_df = pd.DataFrame()
for customer in all_customers:
    cust_df = df[df['Customer_ID'] == customer]
    new_row = detect_all_abnormal_values(cust_df,rules)
    final_df = pd.concat([final_df,new_row],ignore_index=True)

In [99]:
final_df

,ID,Customer_ID,Month,Name,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1608,CUS_0xd40,July,Aaron Maashoh,23.0,Scientist,19114.12,1824.843333,3.0,4.0,...,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.344067,Low_spent_Small_value_payments,244.5653167062043,Good
1,0x1615,CUS_0x21b1,August,Rick Rothackerj,28.0,Teacher,34847.84,3037.986667,2.0,4.0,...,Good,605.03,32.933856,27 Years and 2 Months,No,18.816215,218.904344,Low_spent_Small_value_payments,356.07810855965045,Good
2,0x1621,CUS_0x2dbc,August,Langep,34.0,Engineer,143162.64,12187.220000,1.0,5.0,...,Good,1303.01,38.374753,18 Years and 4 Months,No,246.992319,10000.000000,High_spent_Small_value_payments,895.494583180492,Standard
3,0x1628,CUS_0xb891,March,Jasond,55.0,Entrepreneur,30689.89,2612.490833,2.0,5.0,...,Good,632.46,32.301163,17 Years and 5 Months,NM,16.415452,83.406509,High_spent_Medium_value_payments,411.42712287098345,Standard
4,0x1669,CUS_0x4157,August,Charlie Zhur,24.0,Doctor,114838.41,9843.867500,2.0,5.0,...,Good,1377.74,27.813354,21 Years and 11 Months,No,226.892792,254.571767,High_spent_Large_value_payments,742.9221908300779,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10415,0x25faf,CUS_0x2084,June,Ryanl,21.0,Architect,38321.39,3106.647859,4.0,4.0,...,Good,678.57,37.367063,17 Years and 4 Months,No,362.072453,26.701680,High_spent_Large_value_payments,434.472091,Standard
10416,0x25fc6,CUS_0xf16,May,Maria Sheahanb,1753.0,Media_Manager,16680.35,1528.029167,1.0,1.0,...,Good,897.16,37.788217,21 Years and 5 Months,No,41.113561,64.549748,Low_spent_Large_value_payments,317.139608,Standard
10417,0x25fd3,CUS_0xaf61,June,Chris Wickhamm,50.0,Writer,37188.10,3097.008333,1.0,4.0,...,Good,620.64,37.753013,30 Years and 2 Months,NM,84.205949,147.339908,Low_spent_Small_value_payments,368.154976,Good
10418,0x25fe1,CUS_0x8600,August,Sarah McBridec,29.0,Architect,20002.88,1929.906667,10.0,8.0,...,Bad,3571.7,37.140784,6 Years and 3 Months,Yes,60.964772,34.662906,High_spent_Large_value_payments,337.362988,Standard


In [101]:
df = final_df[final_df["Interest_Rate"].between(0, 100)]

In [102]:
df.to_csv("../data/customer_data.csv")